In [1]:
import pandas as pd
import numpy as np
import scipy
import vincent
import string
import operator
from contextlib import contextmanager
from string import capwords

In [2]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [3]:
crime = pd.read_csv('../train.csv')

In [4]:
crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [8]:
xx = str(crime.X[0])[:6]
xx
yy = str(crime.Y[0])[:4]
yy

'37.7'

In [5]:
X = map(lambda x:double(str(x)[:7]),crime.X)
Y = map(lambda x:double(str(x)[:5]),crime.Y)

In [6]:
xy = [(x,y) for x,y in zip(X,Y)]
len(xy)

878049

In [7]:
crime['xy'] = xy

In [8]:
dummy_xy = pd.get_dummies(crime['xy'],prefix = 'xy')

In [9]:
dates = pd.DatetimeIndex(crime.Dates)


In [12]:
hour = dates[0].hour
crime['hour'] = hour
dummy_hour = pd.get_dummies(crime['hour'],prefix='hour')

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [14]:
dummy_ranks = pd.get_dummies(crime['Descript'], prefix='descript')

In [15]:
len(dummy_ranks.keys())

879

In [16]:
dummy_days = pd.get_dummies(crime['DayOfWeek'],prefix='day')

In [17]:
dummy_district = pd.get_dummies(crime['PdDistrict'],prefix='District')

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score

In [19]:
dummy_resolution = pd.get_dummies(crime['Resolution'],prefix='Res')

In [20]:
clf = MultinomialNB() 
crime_train = dummy_days.join(dummy_district)\
    .join(dummy_hour).join(dummy_resolution).join(dummy_xy) 

In [75]:
#crime_train['hour']=hour

In [36]:
len(crime_train.keys())

896

In [43]:
ctt=0
cats = set(crime.Category)
cat_num = np.arange(len(cats))
cat_dict = {}
train_y=[]
for kk,vv in zip(cats,cat_num):
    cat_dict[kk]=vv
for yy in crime.Category:
    train_y+=[cat_dict[yy]]


In [58]:
labels = cats

In [124]:
k_fold = KFold(n=len(crime_train), n_folds=6)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])

In [133]:
for train_indices, test_indices in k_fold:
    train_X = crime_train.iloc[train_indices]
    train_y = crime.iloc[train_indices]['Category'].values.astype(str)
 
    test_X = crime_train.iloc[test_indices]
    test_y = crime.iloc[test_indices]['Category'].values.astype(str)
    
    clf.fit(train_X, train_y)
    predictions = clf.predict(test_X)
 
    #confusion += confusion_matrix(test_y, predictions,labels)
    score = f1_score(test_y, predictions, pos_label=labels)
    scores.append(score)

/Users/sanghamitra_deb/SanFranciscoCrime/_notebook/lib/python2.7/site-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/Users/sanghamitra_deb/SanFranciscoCrime/_notebook/lib/python2.7/site-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for 

In [49]:
len(predictions)

146342

In [50]:
len(test_X)

146342

In [52]:
len(crime)

878049

In [57]:
predictions.shape

(146342,)

In [56]:
predictions.shape

(146342,)

In [134]:
scores

[0.28962441715209369,
 0.26917592329457402,
 0.27215685308654985,
 0.26733245686023305,
 0.24998456721668469,
 0.24266472496643873,
 0.27399285445082,
 0.25549032949248951,
 0.25769730556947201,
 0.24927446398603334,
 0.21986638336813227,
 0.20578285149720887]

In [68]:
[0.99781650085669027,
 0.99514271956024847,
 0.99649997033482363,
 0.99574101499103096,
 0.99567027134636754,
 0.9944478698560878]

[0.9978165008566903,
 0.9951427195602485,
 0.9964999703348236,
 0.995741014991031,
 0.9956702713463675,
 0.9944478698560878]

In [64]:
crime_test=pd.read_csv('../test.csv')

In [1]:
crime_test.keys()

NameError: name 'crime_test' is not defined

In [66]:
crime_test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
